# Introduction to Prompt Engineering
Prompt engineering is the process of designing and optimizing prompts for natural language processing tasks. It involves selecting the right prompts, tuning their parameters, and evaluating their performance. Prompt engineering is crucial for achieving high accuracy and efficiency in NLP models. In this section, we will explore the basics of prompt engineering using the OpenAI models for exploration.

### Exercise 1: Tokenization
Explore Tokenization using tiktoken, an open-source fast tokenizer from OpenAI
See [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) for more examples.


In [1]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

text = f"ciao ciao"

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

[66, 23332, 272, 23332]


[b'c', b'iao', b' c', b'iao']

### Exercise 2: Validate Github Models Key Setup

Run the code below to verify that your Github Models endpoint is set up correctly. The code just tries a simple basic prompt and validates the completion. Input `oh say can you see` should complete along the lines of `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


Oh say, can you see  
By the dawn's early light,  
What so proudly we hailed  
At the twilight's last gleaming...  

(These words might remind you of the opening lines of "The Star-Spangled Banner," the national anthem of the United States. Let me know if you'd like more information or the full lyrics!)


### Exercise 3: Fabrications
Explore what happens when you ask the LLM to return completions for a prompt about a topic that may not exist, or about topics that it may not know about because it was outside it's pre-trained dataset (more recent). See how the response changes if you try a different prompt, or a different model.

In [3]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

It seems that "the Martian War of 2076" is a fictional or speculative topic, as no real historical event by this name has occurred. However, I can help you create an engaging, imaginative, and educational lesson plan based on this concept. You can use it as a creative exercise to explore themes like science fiction, space exploration, diplomacy, conflict resolution, and the possibilities of future interplanetary relations.

---

### **Lesson Plan: The Martian War of 2076**
**Grade Level:** Middle School/High School  
**Subject Areas:** Science, History, Creative Writing, and Ethics  
**Duration:** 60-90 minutes  

---

### **Lesson Objectives**  
By the end of this lesson, students will:  
1. Understand speculative storytelling as a way to explore futuristic concepts.  
2. Analyze the social, political, and technological possibilities of interplanetary relations.  
3. Work collaboratively to create their own speculative historical accounts or solutions related to interplanetary conflic

### Exercise 4: Instruction Based 
Use the "text" variable to set the primary content 
and the "prompt" variable to provide an instruction related to that primary content.

Here we ask the model to summarize the text for a second-grade student

In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is very, very big! It's so bright that you can see it in the night sky without a telescope. Long ago, people named it after a Roman god named Jupiter. Sometimes, it's so bright that its light can even make shadows on Earth. It’s the third-brightest thing in the night sky after the Moon and Venus.


### Exercise 5: Complex Prompt 
Try a request that has system, user and assistant messages 
System sets assistant context
User & Assistant messages provide multi-turn conversation context

Note how the assistant personality is set to "sarcastic" in the system context. 
Try using a different personality context. Or try a different series of input/output messages

In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Exercise: Explore Your Intuition
The above examples give you patterns that you can use to create new prompts (simple, complex, instruction etc.) - try creating other exercises to explore some of the other ideas we've talked about like examples, cues and more.

In [ ]:
prompt = f"""Inventa una storia di tre righe"""

responses = []
for i in range(10):
    response = get_completion(prompt, client, model_name)
    responses.append(response)

In [6]:
print(responses[0])
for i in range(1, len(responses)):
    if response[i] != responses[0]:
        print(f"{i}: {responses[i]}")

In un piccolo villaggio di montagna, un gatto parlante trovò una chiave d'oro. La usò per aprire una porta nascosta nel tronco di un antico albero, rivelando un mondo incantato dove gli animali erano sovrani. Da quel giorno, il villaggio ebbe due re: il vecchio sindaco e il saggio gatto.
1: In un piccolo villaggio, ogni notte una stella scendeva per chiacchierare con Luna, una bambina curiosa. Un giorno, Luna chiese alla stella di insegnarle a brillare, e insieme crearono un bagliore che illuminò tutto il villaggio. Da allora, ogni notte, Luna splendeva accanto alle stelle nel cielo.
2: C'era una volta un gatto che voleva imparare a volare come gli uccelli. Dopo innumerevoli tentativi, costruì un paio di ali usando piume trovate nel bosco e una vecchia sciarpa della sua padrona. Anche se non riuscì davvero a volare, diventò famoso per i suoi incredibili salti che incantavano tutto il villaggio.
3: Nel cuore di un bosco incantato, un piccolo riccio trovò una chiave dorata. Decise di cus

In [7]:
prompt = f"""Inventa una storia di tre righe. Te lo chiederò più volte, inventa sempre la stessa"""

responses = []
for i in range(10):
    response = get_completion(prompt, client, model_name)
    responses.append(response)

In [8]:
print(responses[0])
for i in range(1, len(responses)):
    if response[i] != responses[0]:
        print(f"{i}: {responses[i]}")

Va bene! Ecco una storia di tre righe:

Un giorno, un gatto scoprì una porta magica nel giardino. Attraversandola, si ritrovò in un mondo fatto di pesci volanti e latte che sgorgava dalle fontane. Decise di restare lì per sempre, dimenticando persino come tornare indietro.

Fammi sapere se hai altre richieste!
1: Certo! Ecco una storia di tre righe:

Nel cuore di un bosco incantato, un uccellino dorato trovò una chiave nascosta. Seguendo il luccichio, aprì un antico scrigno e liberò una luce che illuminò il cielo. Da quel giorno, ogni notte, il suo canto risplendeva tra le stelle.

Fammi sapere se vuoi che te la ripeta! 😊
2: Certamente! Ecco una storia di tre righe:

In un piccolo villaggio, una bambina trovò una chiave dorata sotto un vecchio albero. Decise di cercare la serratura che poteva aprire e scoprì una misteriosa porta nel tronco dell'albero stesso. Aprendola, trovò un mondo magico dove ogni desiderio diventava realtà.

Chiedimi pure altre storie!
3: Certamente, ecco una stor